In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [3]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))


Found GPU at: /device:GPU:0


In [4]:
%ls

datalab/  drive/


In [0]:
import keras
import numpy as np
import os

In [0]:
models_path = 'drive/projects/GOT_LSTM/models'
if not os.path.exists(models_path):
  os.mkdir(models_path)

In [0]:
data_path = 'drive/projects/GOT_LSTM/data'
if not os.path.exists(data_path):
  os.mkdir(data_path)

In [0]:
data_path = os.path.join(data_path, '001ssb.txt')

In [0]:
text = open(path).read().lower()

In [30]:
print(len(text))

1607894


In [31]:
max_len = 100
step = 8
sentences = []
next_chars= [] 

for i in range(0,len(text)-max_len,step):
    sentences.append(text[i:i+max_len])
    next_chars.append(text[i+max_len])
print('number of sequences: ', len(sentences))

number of sequences:  200975


In [32]:
chars = sorted(list(set(text)))
print('unique: ',len(chars))

unique:  53


In [0]:
chars_indices = dict((char, chars.index(char)) for char in chars)

In [0]:
x = np.zeros((len(sentences), max_len,len(chars)), dtype = np.bool)
y = np.zeros((len(sentences), len(chars)), dtype= np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i,t, chars_indices[char]] =1
    y[i, chars_indices[next_chars[i]]] =1

In [0]:
from keras import layers
from keras.models import load_model

In [0]:
model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(max_len, len(chars)),return_sequences= True))
model.add(layers.Dropout(0.3))
model.add(layers.LSTM(128))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(len(chars),activation='softmax'))

In [0]:
model.compile(optimizer= keras.optimizers.RMSprop(lr =0.01), loss = 'categorical_crossentropy')

saved_model = 'LSTM_GOT.h5'
path = os.path.join(models_path, saved_model)
model =load_model(path)

In [0]:
def sample(preds, temperature= 1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)/ temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1,preds, 1)
    return np.argmax(probas)

In [0]:
import random
import sys

In [46]:
callbacks = [keras.callbacks.ModelCheckpoint(filepath=path, monitor='loss', save_best_only= True)]
temp = 0.5

for epoch in range(1,20):
    print('\nepoch', epoch)
    history = model.fit(x,y, batch_size =256 , epochs= 1,callbacks= callbacks,validation_split=0.3)
    start_index = random.randint(0,len(text)-max_len-1)
    generated_text = text[start_index:start_index +max_len]
    print('\n---Generating with seed: "'+ generated_text + '"')
    print('\n-----temperature: ', temp,'\n')
    sys.stdout.write(generated_text)
    for i in range(100):
        sampled = np.zeros((1,max_len, len(chars)))

        for t, char in enumerate(generated_text):
            sampled[0,t,chars_indices[char]] =1
        preds = model.predict(sampled, verbose=0)[0]
        next_index = sample(preds, temp)
        next_char = chars[next_index]

        generated_text += next_char
        generated_text = generated_text[1:]

        sys.stdout.write(next_char)


epoch 1
Train on 140682 samples, validate on 60293 samples
Epoch 1/1
140682/140682 [==============================] - 309s 2ms/step - loss: 1.7338 - val_loss: 1.6423

---Generating with seed: "ds are like swords, i do fear. the old ones go to rust. ah, and here is our milk." the serving girl "

-----temperature:  0.5 

ds are like swords, i do fear. the old ones go to rust. ah, and here is our milk." the serving girl some of the 
nothing to the wamy had feet to were on stark said come lord have a nother, 

stook would
epoch 2
Train on 140682 samples, validate on 60293 samples
Epoch 1/1
140682/140682 [==============================] - 309s 2ms/step - loss: 1.6994 - val_loss: 1.6216

---Generating with seed: "ys was your king, was he not?" 
"he was, my lady." 
"viserys is dead. i am his heir, the last blood "

-----temperature:  0.5 

ys was your king, was he not?" 
"he was, my lady." 
"viserys is dead. i am his heir, the last blood 
the place in the not on the knight in the keave that was it to

 her mording a knight was 
a stell an
epoch 3
Train on 140682 samples, validate on 60293 samples
Epoch 1/1
140682/140682 [==============================] - 313s 2ms/step - loss: 1.6756 - val_loss: 1.6062

---Generating with seed: "nd she was knee-deep in foulsmelling water, wishing she could 
dance upon it as syrio might have, an"

-----temperature:  0.5 

nd she was knee-deep in foulsmelling water, wishing she could 
dance upon it as syrio might have, and

 their should be standing for her that the hound seemed a moron stark of the mountains and strow a 
epoch 4
Train on 140682 samples, validate on 60293 samples
Epoch 1/1
102912/140682 [====================>.........] - ETA: 1:15 - loss: 1.6517

140682/140682 [==============================] - 313s 2ms/step - loss: 1.6571 - val_loss: 1.6072

---Generating with seed: "oot 
king." he clapped his hands together. "a cart! bring cart for khal rhaggat! " 
five thousand do"

-----temperature:  0.5 

oot 
king." he clapped his hands together. "a cart! bring cart for khal rhaggat! " 
five thousand down the came that that." 
"the barrister with the it to him is a said in the storight their fire woul
epoch 5
Train on 140682 samples, validate on 60293 samples
Epoch 1/1
 29440/140682 [=====>........................] - ETA: 3:44 - loss: 1.6370

...

140682/140682 [==============================] - 313s 2ms/step - loss: 1.5390 - val_loss: 1.5437

---Generating with seed: "ded to trick. "how many guards does my father have?" she asked him as they descended to her 
bedcham"

-----temperature:  0.5 

ded to trick. "how many guards does my father have?" she asked him as they descended to her 
bedchame the took to the lade, when the store to the count with his true and dod can the start, his brothe

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


r
epoch 18
Train on 140682 samples, validate on 60293 samples
Epoch 1/1
140682/140682 [==============================] - 313s 2ms/step - loss: 1.5340 - val_loss: 1.5321

---Generating with seed: "daughters of hers, dressed in men's mail. maege is a hoary old 
snark, stubborn, short-tempered, and"

-----temperature:  0.5 

daughters of hers, dressed in men's mail. maege is a hoary old 
snark, stubborn, short-tempered, and ser brad and hamed him. "bran is hand was was gone of 
hear." 
"the man did not well

 her lannisters
epoch 19
Train on 140682 samples, validate on 60293 samples
Epoch 1/1
140682/140682 [==============================] - 312s 2ms/step - loss: 1.5328 - val_loss: 1.5523

---Generating with seed: " oversize head, they found 
a huge bucket-shaped greathelm topped with a foot-long triangular spike."

-----temperature:  0.5 

 oversize head, they found 
a huge bucket-shaped greathelm topped with a foot-long triangular spike. 
"no bare stead with the hall 
gareing of the chil

dren said. some still 
with of the mord said. 
"y

In [94]:
temp =0.475
generated_text = text[200:300]
print('\n---Generating with seed: "'+ generated_text + '"\n')
for i in range(400):
    sampled = np.zeros((1,max_len, len(chars)))
    for t, char in enumerate(generated_text):
        sampled[0,t,chars_indices[char]] =1
    preds = model.predict(sampled, verbose=0)[0]
        
    next_index = sample(preds, temp)
    next_char = chars[next_index]

    generated_text += next_char
    generated_text = generated_text[1:]
    sys.stdout.write(next_char)


---Generating with seed: "o the dead frighten you?" ser waymar royce asked with just the hint of a smile. 
gared did not rise "

she would

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


 get the stone come to the torch. 
"it were the blade still do a can shad the mountace, but we see you warmers and make any was it was had and lord beside the store she had go came and for me. he had the hands the porlast of the lord and fat or the warve and foress to the king was a head to 
the would and some and she took the bran was stark great of catelyn grown the shadows in his think